In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp pctexamples

In [ ]:
#| export
from pct.hierarchy import PCTHierarchy
from pct.environment_processing import EnvironmentProcessingFactory

## PCTExamples Class

The `PCTExamples` class is designed to load a PCTHierarchy configuration file and provide various methods to interact with the hierarchy.

In [ ]:
#| export
class PCTExamples:
    """
    PCTExamples class provides methods to load a PCT hierarchy from a configuration file, 
    summarize the hierarchy, get the configuration, draw the hierarchy, run the hierarchy 
    for a specified number of steps, and close the environment.
    Attributes:
        hierarchy (PCTHierarchy): The loaded PCT hierarchy.
        env (Environment): The environment associated with the PCT hierarchy.
    """

    def __init__(self, config_file, min=True, early_termination=False, history=False, additional_props=None, render = False):
        """
        Initializes the PCTExamples instance by loading the hierarchy from the given configuration file.
        """
        self.config_file = config_file
        self.hierarchy, self.env, self.environment_properties = PCTHierarchy.load_from_file(config_file, min=min, early_termination=early_termination, 
                                                                                            history=history, additional_props=additional_props, render=render)
        self.history_data = None
        self.history = history

    def summary(self):
        """
        Prints a summary of the hierarchy.
        """
        self.hierarchy.summary()

    def get_config(self):
        """
        Returns the configuration of the hierarchy.
        """
        return self.hierarchy.get_config()

    def draw(self, with_labels=True, with_edge_labels=False, font_size=12, font_weight='bold', font_color='black', 
                       color_mapping={'PL':'aqua','OL':'limegreen','CL':'goldenrod', 'RL':'red', 'I':'silver', 'A':'yellow'},
                       node_size=500, arrowsize=25, align='horizontal', file=None, figsize=(8,8), move={}, draw_fig=True,
                       node_color=None, layout={'r':2,'c':1,'p':2, 'o':0}, funcdata=False, interactive_mode=False, experiment=None):
        """
        Draws the hierarchy with the specified parameters and returns the figure.
        """
        fig = self.hierarchy.draw(with_labels, with_edge_labels, font_size, font_weight, font_color, color_mapping, node_size, 
                            arrowsize, align, file, figsize, move, draw_fig, node_color, layout, funcdata, interactive_mode, experiment)
        
        return fig

    def run(self, steps=1, verbose=False):#, render=False):
        """
        Runs the hierarchy for the specified number of steps and returns the result.
        """
        # self.hierarchy.get_preprocessor()[0].set_render(render)
        return self.hierarchy.run(steps, verbose)

    def results(self, verbose=False, args=None, environment_properties=None):
        print(self.environment_properties)
        env_name = self.env.get_name()
        env_proc = EnvironmentProcessingFactory.createEnvironmentProcessing(f'{env_name}EnvironmentProcessing')
        allargs={'drive':'', 'verbosed':{'hpct_verbose':verbose}, 'max': False} | args
        env_proc.set_properties(allargs)
        env_proc.results(filepath=self.config_file, environment_properties=self.environment_properties|environment_properties, hierarchy=self.hierarchy)


    def close(self):
        """
        Closes the environment.
        """
        self.env.close()

    def get_history_keys(self):
        """
        Returns the keys of the history data of the hierarchy.
        """
        if self.history :
            if self.history_data is None:
                self.history_data = self.hierarchy.get_history_data()
            return list(self.history_data.keys())
        else:
            print("History is not enabled")

        return None

    def set_history_data(self):
        """
        Sets the history data of the hierarchy.
        """
        if self.history :

            if self.history_data is None:
                self.history_data = self.hierarchy.get_history_data()

            return self.history_data
        else:
            print("History is not enabled")

        return None

    def get_config_file_contents(self):
        """
        Reads and returns the contents of the config file.
        """
        with open(self.config_file, 'r') as file:
            return file.read()

    def plot_history(self, plots=None, title_prefix='', plots_dir=None, plots_figsize=(12, 6), history_data=None):
        """
        Plots the history of the hierarchy.
        """
        print(plots)
        plots = self.hierarchy.get_plots_config(plots, title_prefix)
        print(plots)

        from os import sep
        for plot in plots:
            print(plot)
            plotfile=None
            if plots_dir:
                plotfile = plots_dir + sep + plot['title'] + '-' + str(self.hierarchy.get_namespace()) + '.png'
            fig = self.hierarchy.hierarchy_plots(title=plot['title'], plot_items=plot['plot_items'], figsize=plots_figsize, file=plotfile, history=history_data)
            # import matplotlib.pyplot as plt
            # plt.close(fig)  # Close the figure here
    
        return fig
    
    def plot_single(self, plot=None, title_prefix='', plots_dir=None, plots_figsize=(12, 6), history_data=None):
        """
        Plot one item of the history of the hierarchy.
        """

        from os import sep, path, makedirs

        plotfile=None
        if plots_dir:
            if not path.exists(plots_dir):
                makedirs(plots_dir)
            plotfile = plots_dir + sep + title_prefix + plot['title'] + '-' + str(self.hierarchy.get_namespace()) + '.png'
        fig = self.hierarchy.hierarchy_plots(title=title_prefix + plot['title'], plot_items=plot['plot_items'], figsize=plots_figsize, file=plotfile, history=history_data)
            # import matplotlib.pyplot as plt
            # plt.close(fig)  # Close the figure here
    
        return fig
    
    def run_from_config(self, config):
        """
        Run the hierarchy with configuration options.
        
        Args:
            config (dict): Configuration dictionary with options:
                - steps (int): Number of timesteps to run (default: 1000)
                - verbose (bool): Enable logging for debugging (default: False)
                - render (bool): Render the environment during execution (default: False)
                - video_file (str): Save video recording to specified file path
                - draw_file (str): Export hierarchy visualization to given path
                - size (bool): Return the size of the PCT network (default: False)
                - usage (bool): Print help message explaining config options (default: False)
        
        Returns:
            dict: Results containing any requested outputs (size, etc.)
        """
        results = {}
        
        # Print usage if requested
        if config.get('usage', False):
            self._print_usage()
            return results
        
        # Get configuration values with defaults
        steps = config.get('steps', 1000)
        verbose = config.get('verbose', False)
        render = config.get('render', False)
        video_file = config.get('video_file')
        draw_file = config.get('draw_file')
        return_size = config.get('size', False)
        
        # Calculate and return size if requested
        if return_size:
            size_info = self._calculate_size()
            results['size'] = size_info
            if verbose:
                print(f\"PCT Network Size: {size_info}\")
        
        # Set up video recording if requested
        original_env = None
        if video_file:
            try:
                from gym.wrappers import Monitor
                import os
                # Create video directory if it doesn't exist
                video_dir = os.path.dirname(video_file) or './video'
                os.makedirs(video_dir, exist_ok=True)
                # Wrap environment for video recording
                original_env = self.env
                self.env = Monitor(self.env, video_dir, force=True, video_callable=lambda episode_id: True)
                if verbose:
                    print(f\"Video recording enabled: {video_file}\")
            except ImportError:
                if verbose:
                    print(\"Warning: gym.wrappers.Monitor not available, skipping video recording\")
            except Exception as e:
                if verbose:
                    print(f\"Warning: Could not set up video recording: {e}\")
        
        # Run the hierarchy
        if verbose:
            print(f\"Running hierarchy for {steps} steps with verbose={verbose}, render={render}\")
        
        try:
            result = self.run(steps=steps, verbose=verbose)
            results['run_result'] = result
            
            if verbose:
                print(f\"Hierarchy run completed successfully\")
                
        except Exception as e:
            if verbose:
                print(f\"Error during hierarchy run: {e}\")
            results['error'] = str(e)
        
        # Restore original environment if video recording was used
        if original_env is not None:
            self.env.close()
            self.env = original_env
            
            # Rename video file if custom name was specified
            if video_file and video_file != './video':
                try:
                    import glob
                    import shutil
                    video_files = glob.glob(os.path.join(video_dir, '*.mp4'))
                    if video_files:
                        latest_video = max(video_files, key=os.path.getctime)
                        if os.path.dirname(video_file) != video_dir:
                            shutil.move(latest_video, video_file)
                            if verbose:
                                print(f\"Video saved to: {video_file}\")
                        else:
                            if verbose:
                                print(f\"Video saved to: {latest_video}\")
                except Exception as e:
                    if verbose:
                        print(f\"Warning: Could not move video file: {e}\")
        
        # Generate drawing if requested
        if draw_file:
            try:
                fig = self.draw(file=draw_file)
                results['draw_result'] = f\"Drawing saved to: {draw_file}\"
                if verbose:
                    print(f\"Hierarchy drawing saved to: {draw_file}\")
            except Exception as e:
                if verbose:
                    print(f\"Warning: Could not save drawing: {e}\")
                results['draw_error'] = str(e)
        
        return results
    
    def _calculate_size(self):
        \"\"\"Calculate the size of the PCT network.\"\"\"
        size_info = {
            'levels': self.hierarchy.get_levels(),
            'total_nodes': 0,
            'nodes_per_level': []
        }
        
        for level in range(self.hierarchy.get_levels()):
            level_nodes = self.hierarchy.get_columns(level)
            size_info['nodes_per_level'].append(level_nodes)
            size_info['total_nodes'] += level_nodes
            
        return size_info
    
    def _print_usage(self):
        \"\"\"Print usage information for the run_from_config method.\"\"\"
        usage_text = \"\"\"
PCT Examples Configuration Options:

Configuration dictionary keys:
  steps (int):       Number of timesteps to run the hierarchy (default: 1000)
  verbose (bool):    Enable logging for debugging and insight (default: False)
  render (bool):     Render the environment during execution (default: False)
  video_file (str):  Save a video recording of the run to the specified file path
  draw_file (str):   Export a visualization of the hierarchy to the given path
  size (bool):       Return the size (number of nodes, levels) of the PCT network (default: False)
  usage (bool):      Print this help message (default: False)

Example usage:
  config = {
      'steps': 500,
      'verbose': True,
      'render': True,
      'video_file': './my_run.mp4',
      'draw_file': './hierarchy.png',
      'size': True
  }
  results = pct_example.run_from_config(config)
        \"\"\"
        print(usage_text)

    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()